# 🥈 Camada Silver — Openbrewery

A camada Silver é responsável por **transformar, validar e garantir a qualidade dos dados** provenientes da Bronze.

Enquanto a Bronze armazena os dados brutos exatamente como chegaram, a Silver aplica:

- Padronização de tipos e formatos
- Regras de Data Quality
- Separação entre dados válidos e inválidos (quarantine)
- Deduplicação
- Carga incremental na tabela Delta final

## 🔄 Fluxo da Silver

O processamento da Silver segue 4 etapas bem definidas:

1. **Staging** — padronização e tipagem dos dados
2. **Validação (Data Quality)** — separação entre registros válidos e inválidos
3. **Quarantine** — armazenamento dos registros com erro para análise
4. **Merge incremental** — carga apenas dos registros válidos na tabela Silver

In [0]:
USE CATALOG `bees-teste-jp`;
USE SCHEMA silver;

## 🧪 1. Staging — Padronização dos dados

A view `vw_openbrewery_staging` prepara os dados vindos da Bronze:

- Conversão de tipos (CAST)
- Padronização de textos (`TRIM`, `LOWER`)
- Tratamento de nulos (`COALESCE`)
- Criação de coluna técnica `updated_at`

Essa etapa ainda **não aplica regras de qualidade**, apenas prepara os dados.

In [0]:
CREATE OR REPLACE TEMP VIEW vw_openbrewery_staging AS
SELECT 
    CAST(id AS STRING) AS brewery_id,
    TRIM(name) AS name,
    LOWER(brewery_type) AS brewery_type,
    city,
    COALESCE(state_province, 'Unknown') AS state_province, 
    country,
    CAST(longitude AS DOUBLE) AS longitude,
    CAST(latitude AS DOUBLE) AS latitude,
    from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo') AS updated_at
FROM  `bees-teste-jp`.bronze.openbrewery
WHERE id IS NOT NULL;

## ✅ 2. Data Quality — Registros válidos

A view `vw_openbrewery_valid` contém apenas os registros que atendem às regras de qualidade:

### Regras aplicadas

| Regra | Dimensão de Qualidade |
|---|---|
| `name` não nulo | Completeness |
| `brewery_type` não nulo | Completeness |
| `country` não nulo | Completeness |
| Latitude entre -90 e 90 | Validity |
| Longitude entre -180 e 180 | Validity |

Somente esses dados podem seguir para a Silver.

In [0]:
CREATE OR REPLACE TEMP VIEW vw_openbrewery_valid AS
SELECT *
FROM vw_openbrewery_staging
WHERE
    latitude IS NOT NULL
    AND longitude IS NOT NULL
    AND latitude BETWEEN -90 AND 90
    AND longitude BETWEEN -180 AND 180;

## ❌ 3. Data Quality — Registros inválidos (Quarantine)

A view `vw_openbrewery_invalid` identifica os registros que falharam nas regras e adiciona a coluna `dq_error` com o motivo da falha.

Esses registros **não são descartados**.

Eles são inseridos na tabela: `quarantine_openbrewery`

In [0]:
CREATE OR REPLACE TEMP VIEW vw_openbrewery_invalid AS
SELECT *,
  CASE
      WHEN (latitude IS NULL AND longitude IS NULL) THEN 'Latitude e longitude nula'
      WHEN latitude IS NULL THEN 'Latitude nula'
      WHEN longitude IS NULL THEN 'Longitude nula'
      WHEN latitude NOT BETWEEN -90 AND 90 THEN 'Latitude invalida'
      WHEN longitude NOT BETWEEN -180 AND 180 THEN 'Longitude invalida'
      WHEN name IS NULL THEN 'Nome nulo'
      WHEN brewery_type IS NULL THEN 'Tipo de cervejaria nula'
      WHEN country IS NULL THEN 'Pais nulo'
  END AS dq_error
FROM vw_openbrewery_staging
WHERE NOT (
    latitude IS NOT NULL
    AND longitude IS NOT NULL
    AND latitude BETWEEN -90 AND 90
    AND longitude BETWEEN -180 AND 180
);

## 🗃️ 4. Criação da tabela de Quarantine

A tabela é criada automaticamente com o mesmo schema da view de inválidos usando:

In [0]:
CREATE TABLE IF NOT EXISTS quarantine_openbrewery AS SELECT * FROM vw_openbrewery_invalid WHERE 1=2;

In [0]:
INSERT INTO quarantine_openbrewery
SELECT * FROM vw_openbrewery_invalid;

## 🧩 5. Tabela Silver

A tabela `silver_openbrewery` é criada em formato Delta e particionada por `state_province`.

Essa tabela contém apenas dados limpos e confiáveis.

---

## 🔁 6. Merge Incremental

O carregamento na Silver é feito com `MERGE`, garantindo:

- Atualização de registros existentes
- Inserção de novos registros
- Deduplicação via `ROW_NUMBER`
- Processamento incremental

A Silver nunca é recriada, apenas atualizada.

In [0]:
CREATE TABLE IF NOT EXISTS openbrewery (
    brewery_id STRING,
    name STRING,
    brewery_type STRING,
    city STRING,
    state_province STRING,
    country STRING,
    longitude DOUBLE,
    latitude DOUBLE,
    updated_at TIMESTAMP
)
USING DELTA
PARTITIONED BY (state_province);

MERGE INTO openbrewery AS target
USING (
    SELECT * FROM (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY brewery_id ORDER BY updated_at DESC) as rn
        FROM vw_openbrewery_valid
    ) WHERE rn = 1
) AS source
ON target.brewery_id = source.brewery_id

WHEN MATCHED THEN
  UPDATE SET 
    target.name = source.name,
    target.brewery_type = source.brewery_type,
    target.city = source.city,
    target.state_province = source.state_province,
    target.country = source.country,
    target.longitude = source.longitude,
    target.latitude = source.latitude,
    target.updated_at = source.updated_at

WHEN NOT MATCHED THEN
  INSERT (
    brewery_id, 
    name, 
    brewery_type, 
    city, 
    state_province, 
    country, 
    longitude, 
    latitude, 
    updated_at
  )
  VALUES (
    source.brewery_id, 
    source.name, 
    source.brewery_type, 
    source.city, 
    source.state_province, 
    source.country, 
    source.longitude, 
    source.latitude, 
    source.updated_at
  );